<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_21_CrewAI_Medical_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to use Graphlit with CrewAI. Based on this [great blog post](https://farzzy.hashnode.dev/building-healthcare-ai-agent-medical-guidelines) and [notebook](https://farzzy.hashnode.dev/building-healthcare-ai-agent-medical-guidelines) written by Farzad Sunavala, we emulate Farzad's workflow and show how to use Graphlit instead of LlamaIndex, while already leveraging Azure AI Search under the hood.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

For CrewAI, you will need your OpenAI API key.

Assign this property as Colab secret: OPENAI_API_KEY

---

In [1]:
!pip install --upgrade crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade graphlit-tools[crewai]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 2.2 MB/s eta 0:00:00


Clone the original repo to access the PDFs

In [3]:
!rm -rf azure-ai-search-python-playground
!git clone https://github.com/farzad528/azure-ai-search-python-playground

Cloning into 'azure-ai-search-python-playground'...
remote: Enumerating objects: 1068, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 1068 (delta 58), reused 100 (delta 41), pack-reused 947 (from 1)
Receiving objects: 100% (1068/1068), 289.17 MiB | 16.36 MiB/s, done.
Resolving deltas: 100% (204/204), done.
Updating files: 100% (96/96), done.


Initialize Graphlit

In [4]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Initialize sample data

In [5]:
DATA_DIRECTORY = "azure-ai-search-python-playground"

PATH_PDFS = DATA_DIRECTORY + "/data/acc/"

Define Graphlit helper functions

In [6]:
import base64
import mimetypes
import asyncio
import time
from typing import List, Optional
from tqdm.asyncio import tqdm

async def process_file(filename, progress_bar):
    pdf_path = os.path.join(PATH_PDFS, filename)

    print(f'Starting to ingest content from [{pdf_path}].')

    content_id = await ingest_file(pdf_path)

    if content_id is not None:
        print(f'Completed ingesting content [{content_id}] from [{pdf_path}].')

    progress_bar.update(1)

# NOTE: for local files, load from disk and convert to Base64 data
async def ingest_file(file_path: str):
    if graphlit.client is None:
        return;

    try:
        file_name = os.path.basename(file_path)
        content_name, _ = os.path.splitext(file_name)

        mime_type = mimetypes.guess_type(file_name)[0]

        if mime_type is None:
            print('Failed to infer MIME type')
            return None

        with open(file_path, "rb") as file:
            file_content = file.read()

        base64_content = base64.b64encode(file_content).decode('utf-8')

        response = await graphlit.client.ingest_encoded_file(content_name, base64_content, mime_type, is_synchronous=True)

        return response.ingest_encoded_file.id if response.ingest_encoded_file is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model,
        ),
        strategy=input_types.ConversationStrategyInput(
            embedCitations=True
        ),
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.SECTION
        ),
        rerankingStrategy=input_types.RerankingStrategyInput(
            serviceType=enums.RerankingModelServiceTypes.COHERE
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return None, None

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        message = response.prompt_conversation.message.message if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
        citations = response.prompt_conversation.message.citations if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None

        return message, citations
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None, None

# NOTE: these functions are just used to clean-up old data before executing the example
async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

Ingest sample PDFs

In [7]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

# List of files to process
files = os.listdir(PATH_PDFS)

progress_bar = tqdm(total=len(files))

tasks = [process_file(filename, progress_bar) for filename in files]

await asyncio.gather(*tasks)

print('Ingested all contents.')

Deleted all contents.


  0%|          | 0/36 [00:00<?, ?it/s]

Starting to ingest content from [azure-ai-search-python-playground/data/acc/shen-et-al-2017-2017-acc-aha-hrs-guideline-for-the-evaluation-and-management-of-patients-with-syncope.pdf].
Starting to ingest content from [azure-ai-search-python-playground/data/acc/epstein-et-al-2008-acc-aha-hrs-2008-guidelines-for-device-based-therapy-of-cardiac-rhythm-abnormalities.pdf].
Starting to ingest content from [azure-ai-search-python-playground/data/acc/halperin-et-al-2015-further-evolution-of-the-acc-aha-clinical-practice-guideline-recommendation-classification-system.pdf].
Starting to ingest content from [azure-ai-search-python-playground/data/acc/o-gara-et-al-2012-2013-accf-aha-guideline-for-the-management-of-st-elevation-myocardial-infarction.pdf].
Starting to ingest content from [azure-ai-search-python-playground/data/acc/grundy-et-al-2018-2018-aha-acc-aacvpr-aapa-abc-acpm-ada-ags-apha-aspc-nla-pcna-guideline-on-the-management-of-blood.pdf].
Starting to ingest content from [azure-ai-search-py

  6%|▌         | 2/36 [00:20<04:42,  8.32s/it]

Completed ingesting content [9441e59b-e072-404d-8f0e-624d7358a37d] from [azure-ai-search-python-playground/data/acc/arnett-et-al-2014-aha-acc-hhs-strategies-to-enhance-application-of-clinical-practice-guidelines-in-patients-with.pdf].
Completed ingesting content [788c2c3a-9804-4c80-bfed-c19b28bc3482] from [azure-ai-search-python-playground/data/acc/anderson-et-al-2013-management-of-patients-with-peripheral-artery-disease-(compilation-of-2005-and-2011-accf-aha.pdf].


...

Completed ingesting content [6c8ba084-7c12-4238-917b-83b266c960f9] from [azure-ai-search-python-playground/data/acc/heidenreich-et-al-2022-2022-aha-acc-hfsa-guideline-for-the-management-of-heart-failure.pdf].


 97%|█████████▋| 35/36 [02:18<00:03,  3.49s/it]

Completed ingesting content [05689dd5-b03b-4f8d-a62d-9c9ed0156997] from [azure-ai-search-python-playground/data/acc/otto-et-al-2020-2020-acc-aha-guideline-for-the-management-of-patients-with-valvular-heart-disease.pdf].


100%|██████████| 36/36 [02:19<00:00,  2.61s/it]

Completed ingesting content [4ceca09d-64a1-4214-bebb-8509e880a2dd] from [azure-ai-search-python-playground/data/acc/al-khatib-et-al-2018-2017-aha-acc-hrs-guideline-for-management-of-patients-with-ventricular-arrhythmias-and-the.pdf].
Ingested all contents.


Copied from original notebook, for patient configuration

In [9]:
# Define complex patient profile
patient_profile = {
    "patient_id": "43454357890",
    "name": "Sarah Johnson",
    "age": 68,
    "gender": "Female",
    "height_cm": 165,
    "weight_kg": 72,
    "bmi": 26.4,
    "blood_type": "A-",
    "allergies": ["Sulfa drugs"],
    "current_medications": [
        "Flecainide",
        "Apixaban",
        "Metoprolol",
        "Atorvastatin",
        "Cilostazol",
    ],
    "chronic_conditions": {
        "atrial_fibrillation": True,
        "peripheral_arterial_disease": True,
        "hypertension": True,
        "hyperlipidemia": True,
        "coronary_artery_disease": False,
    },
    "family_medical_history": {"heart_disease": True, "stroke": True, "cancer": False},
    "lifestyle_factors": {
        "smoking": "Former smoker, quit 5 years ago",
        "alcohol_use": "Rare",
        "physical_activity_per_week": "4-5 days",
        "diet": "Heart-healthy, low-sodium",
        "sleep_hours_per_night": 6,
    },
    "recent_lab_results": {
        "inr": 1.4,
        "creatinine_mg_dl": 1.1,
        "ldl_cholesterol_mg_dl": 85,
        "hdl_cholesterol_mg_dl": 55,
        "blood_pressure_mm_hg": "135/80",
    },
    "vaccination_status": {
        "influenza_vaccine": True,
        "covid_vaccine": True,
        "pneumonia_vaccine": True,
    },
    "surgical_history": [
        "Carotid endarterectomy",
        "Cholecystectomy",
    ],
    "imaging_history": {
        "last_echocardiogram_date": "2024-07-15",
        "last_carotid_ultrasound_date": "2024-06-20",
    },
    "mental_health": {
        "anxiety": True,
        "depression": False,
        "cognitive_function_issues": False,
    },
    "preferences": {
        "preferred_treatment_type": [
            "Minimally invasive procedures",
            "Evidence-based treatments",
        ],
        "end_of_life_care": "Yes",
        "pain_management": "Non-opioid",
    },
    "recent_visits": [
        {
            "visit_date": "2024-08-25",
            "reason": "Follow-up for atrial fibrillation",
            "notes": "Recurrence of AF despite flecainide. Discussing ablation vs alternative antiarrhythmic options.",
        },
        {
            "visit_date": "2024-07-10",
            "reason": "Peripheral arterial disease management",
            "notes": "Stable claudication symptoms. Continue current management and exercise program.",
        },
    ],
    "af_management": {
        "current_treatment": "Flecainide",
        "treatment_history": [
            "Rate control with metoprolol",
            "Rhythm control with flecainide",
        ],
        "af_recurrence": True,
        "considering_options": ["Catheter ablation", "Alternative antiarrhythmic drug"],
        "chads2_vasc_score": 4,
        "has_bled_score": 2,
    },
    "pad_management": {
        "ankle_brachial_index": 0.78,
        "fontaine_classification": "Stage II",
        "current_treatment": ["Cilostazol", "Supervised exercise program"],
        "last_vascular_assessment": "2024-06-20",
    },
}

Execute example RAG prompt with citations

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()

print('Deleted all conversations and specifications.')

# Formulate a query based on Sarah Johnson's profile
prompt = (
    f"Which option reduces stroke risk for a {patient_profile['age']}-year-old "
    f"female with atrial fibrillation and peripheral arterial disease, "
    f"who is considering catheter ablation or antiarrhythmic drug therapy?"
)

model = enums.OpenAIModels.GPT4O_MINI_128K
#model = enums.OpenAIModels.O1_MINI_128K

specification_id = await create_openai_specification(model)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    conversation_id = await create_conversation(specification_id=specification_id)

    if conversation_id is not None:
        message, citations = await prompt_conversation(conversation_id, prompt)

        if message is not None:
            display(Markdown(f'### Patient-Specific Question Response for {patient_profile["name"]}:'))
            display(Markdown(f'**Final Response:**\n{message}'))
            print()

        if citations is not None:
            print("\nReference Information:")

            for citation in citations:
                if citation is not None and citation.content is not None:
                    display(Markdown(f'**Citation [{citation.index}]:** {citation.content.name}'))
                    display(Markdown(citation.text))
                    print()


Deleted all conversations and specifications.
Created specification [0752a4b9-3cf0-4770-b2ed-730ae38c3e86].


### Patient-Specific Question Response for Sarah Johnson:

**Final Response:**
For a 68-year-old female with atrial fibrillation (AF) and peripheral arterial disease, the primary option to reduce stroke risk is anticoagulation therapy. Direct-acting oral anticoagulants (DOACs) are recommended as the first-line option for patients with AF, regardless of their CHA2DS2-VASc score, which assesses stroke risk. Additionally, if catheter ablation is being considered, it can be effective in managing AF, but the decision should also factor in the patient's overall health and preferences regarding antiarrhythmic drug therapy. [1][2][3][4][5]



Reference Information:


**Citation [1]:** ommen-et-al-2024-2024-aha-acc-amssm-hrs-paces-scmr-guideline-for-the-management-of-hypertrophic-cardiomyopathy

...

8.4. Management of Patients With HCM and AF
Recommendations for Management of Patients With HCM and AF Referenced studies that support the recommendations are summarized in the Online Data Supplement.
| COR | LOE | RECOMMENDATIONS |
| --- | --- | --- |
| 1 | B-NR | 1. In patients with HCM and clinical AF, anticoagulation is recommended with direct-acting oral anticoagu- lants (DOACs) as first-line option and vitamin K antagonists as second-line option, independent of CHA2DS2-VASc score.1-5 |
| 1 | C-LD | 2. In patients with HCM and subclinical AF detected by internal or external cardiac device or monitor of >24 hours' duration for a given episode, anticoagulation is recommended with DOACs as first-line option and vitamin K antagonists as second-line option, independent of CHA2DS2-VASc score.1,6-8 |
| 1 | C-LD | 3. In patients with AF in whom rate control strategy is planned, beta blockers, verapamil, or diltiazem are recommended, with the choice of agents according to patient preferences and comorbid conditions.9,10 |
| 2a | C-LD | 4. In patients with HCM and subclinical AF detected by internal or external device or monitor, of >5 minutes' duration but <24 hours' duration for a given episode, anticoagulation with DOACs as first-line option and vitamin K antagonists as second-line option can be beneficial, taking into consideration duration of AF episodes, total AF burden, underlying risk factors, and bleeding risk.1,6-8,11 |
| 2a | B-NR | 5. In patients with HCM and poorly tolerated AF, a rhythm-control strategy with cardioversion or antiar- rhythmic drugs can be beneficial with the choice of an agent according to AF symptom severity, patient preferences, and comorbid conditions.9,12-24 |
| 2a | B-NR | 6. In patients with HCM and symptomatic AF, as part of an AF rhythm- control strategy, catheter ablation for AF can be effective when drug therapy is ineffective, contraindicated, or not the patient's preference. 12,25,26 |
| 2a | B-NR | 7. In patients with HCM and AF who require surgical myectomy, concomitant surgical AF ablation procedure can be beneficial for AF rhythm control.13,27-29 |


**Citation [4]:** page-et-al-2015-2015-acc-aha-hrs-guideline-for-the-management-of-adult-patients-with-supraventricular-tachycardia

7.4. Ongoing Management: Recommendations
Recommendations for Ongoing Management of Atrial Flutter
| COR | LOE | RECOMMENDATIONS |
| --- | --- | --- |
| I | B-R | 1. Catheter ablation of the CTI is useful in patients with atrial flutter that is either symptomatic or refractory to pharmacological rate control (243,372-375). |
| See Online Data 16 | Supplements and 17. | Rate control can be difficult to achieve in atrial flutter, and a rhythm control strategy is often chosen. Catheter ablation of CTI-dependent atrial flutter is often preferred to long-term pharmacological therapy; in this rhythm, the CTI represents the optimal target for ablation because a line of ablation between the tricuspid valve annulus and inferior vena cava can effectively interrupt the circuit. A variety of techniques are applicable, including various catheter types, energy delivery systems, and mapping and visualization tools; generally, success depends on creation of a complete line of block and permanent interruption of conduction across the CTI (Table 8). Successful ablation is often heralded by intraprocedural interruption of the arrhythmia and subsequent EP demonstration of bidirectional block across the ablated tissue. |
| I | C-LD | 2. Beta blockers, diltiazem, or verapamil are useful to control the ventricular rate in patients with hemodynamically tolerated atrial flutter (347-349). |
| See Online Data 16 | and 17. Supplements | In some circumstances of persistent atrial flutter or in patients who have infrequent reasonably well-tolerated episodes of atrial flutter, a rate-control strategy may be chosen. In atrial flutter, the relatively slower atrial rate compared with AF often paradoxically results in more rapid AV nodal conduction because there is less concealed AV nodal conduction. Therefore, achieving adequate rate control can be difficult. Higher doses of beta blockers, diltiazem, or verapamil, and often a combination of agents, may be needed to achieve adequate rate control. Beta blockers are generally preferred in patients with heart failure. Avoidance of beta blockers, diltiazem, and verapamil in patients with pre-excited atrial flutter is recommended, given the potential for accelerated ventricular rates degenerating to ventricular fibrillation, as has been reported to occur rarely in similar patients with AF (274). |
| I | C-LD | 3. Catheter ablation is useful in patients with recurrent symptomatic non-CTI-dependent flutter after failure of at least 1 antiarrhythmic agent (134,327). |
| See Online Data 16 | and 17. Supplements | No prospective RCTs have compared the efficacy or safety of antiarrhythmic drugs with that of catheter ablation for non-CTI-dependent atrial flutter. In general, catheter ablation of non-CTI-dependent atrial flutter is substantially more difficult than ablation of CTI-dependent flutter because the anatomic circuits are complex, are often not anatomically defined, and can be difficult to locate. Knowledge of the prior surgical or ablation approach and detailed activation and entrainment mapping of the tachycardia are useful during attempts at ablation (Table 8). The location of the circuit determines ablation approach and risks. Observational data support the relative effectiveness and safety of catheter ablation in experienced centers (134,327). Many of the atrial flutters that are observed during the first 3 months after catheter ablation or cardiac surgery will not persist beyond the periprocedural period, so attempts at ablation can be deferred unless pharmacological therapy and/or cardioversion are unsuccessful (327). |
| I | B-NR | 4. Ongoing management with antithrombotic therapy is recommended in patients with atrial flutter to align with recommended antithrombotic therapy for patients with AF (365). |
| See Online Data 16 | Supplements and 17. | The risk of stroke associated with AF is well established, and several RCTs have demonstrated the efficacy of anticoagulation for stroke prevention in patients with additional risk factors (366). Anticoagulation has also been shown to prevent stroke when administered during the weeks immediately before and after cardioversion (367). Whether atrial flutter carries a risk of stroke similar to that in patients with AF has been debated in the past but is now supported by limited evidence from mechanistic, observational, and prospective studies that include patients primarily with atrial flutter (365-369). Several reports have suggested that the risk of stroke in patients with atrial flutter is mitigated by anticoagulation. Meta-analysis of 13 studies of patients undergoing cardioversion of atrial flutter reported short-term stroke risks ranging from 0% to 7%, and the thromboembolism rate in patients with sustained flutter in 4 studies averaged 3% annually (365). Other studies have reported similar risk and effectiveness of anticoagulation (369). Therefore, on the basis of available data, recommendations for antithrombotic therapy for patients with atrial flutter are similar to those for patients with AF (10). |

IIa
B-R
1. The following drugs can be useful to maintain sinus rhythm in patients with symptomatic, recurrent atrial flutter, with the drug choice depending on underlying heart disease and comorbidities:
a. Amiodarone (376)
b. Dofetilide (346,377)
c. Sotalol (378)
See Online Data Supplements 16 and 17.
In patients in whom ablation is not being considered because of contraindications (such as underlying medical illness) or because of patient preference, a variety of antiarrhythmic drugs are available. These drugs act by suppressing triggers and altering atrial tissue refractoriness. Individual properties of each drug need to be considered for proper use. Much of the data pertaining to use of amiodarone in management of atrial arrhythmias has been derived from use in patients with AF(10). Few data are available for patients with atrial flutter. Amiodarone has significant toxicities, so it is used only when other treatments are contraindicated or ineffective. Nevertheless, administration is reasonable, particularly in patients with heart failure or significant underlying heart disease (376).
Dofetilide may be more effective than many other drugs but must be started in an inpatient setting (346,377). The dose is adjusted on the basis of renal function, with close monitoring of the Q-T interval and subsequent monitoring for altered renal function.
Sotalol, a class III antiarrhythmic, is generally well tolerated but is associated with typical beta blocker side effects, such as fatigue and bradycardia (378). The major potential cardiac toxicity for both drugs is torsades de pointes.
IIa
B-NR
2. Catheter ablation is reasonable in patients with CTI-dependent atrial flutter that occurs as the result of flecainide, propafenone, or amiodarone used for treatment of AF (317,379-381).
See Online Data Supplements 16 and 17.
Some patients with AF treated with propafenone, flecainide, or amiodarone will develop atrial flutter. In this circumstance, if flutter becomes the dominant arrhythmia, ablation of the CTI and continued use of the antiarrhythmic drug can decrease the incidence of atrial flutter and facilitate the pharmacological management of AF (379,380).
IIa
C-LD
3. Catheter ablation of the CTI is reasonable in patients undergoing catheter ablation of AF who also have a history of documented clinical or induced CTI-dependent atrial flutter (381,382).
See Online Data Supplements 16 and 17.
The indications for catheter ablation of AF are discussed in the 2014 AF guideline (10). When AF and atrial flutter coexist, 1 randomized study demonstrated that at 1-year follow-up, greater success in terms of arrhythmia suppression and quality-of-life score resulted from AF ablation (with or without atrial flutter ablation) than from atrial flutter ablation alone (381). It may be that AF ablation alone is sufficient to control both arrhythmias, although CTI ablation reduced the early postablation arrhythmia recurrence rate (382).
IIa
C-LD
4. Catheter ablation is reasonable in patients with recurrent symptomatic non-CTI-dependent flutter as primary therapy, before therapeutic trials of antiarrhythmic drugs, after carefully weighing potential risks and benefits of treatment options (135).
See Online Data Supplement 17.
No prospective RCTs have compared the efficacy or safety of antiarrhythmic drugs with that of catheter ablation for non-CTI-dependent atrial flutter. Observational data, however, support the relative effectiveness and safety of catheter ablation in experienced centers (135,327). Many of the atrial flutters that are observed during the first 3 months after ablation or cardiac surgery will not persist beyond the periprocedural period, so attempts at ablation can be deferred unless attempts at pharmacological therapy or cardioversion are unsuccessful (135).
IIb
B-R
1. Flecainide or propafenone may be considered to maintain sinus rhythm in patients without structural heart disease or ischemic heart disease who have symptomatic recurrent atrial flutter (383-385).
See Online Data Supplements 16 and 17.
Data to support the recommendation for flecainide and propafenone for maintenance of sinus rhythm in patients with atrial flutter is derived from trials that pooled patients with AF and atrial flutter, with the vast majority of the patients having AF. It is therefore possible that these agents are not as effective for treating isolated atrial flutter as they are for AF (386). Flecainide and propafenone may result in slowing of the atrial flutter cycle length, which may lead to a rapid 1:1 ventricular response (387). Because of this, caution is advised with flecainide and propafenone in patients with atrial flutter at risk of 1:1 conduction. The risk may be reduced by coadministration of medications that slow AV nodal conduction, such as beta blockers, verapamil, or diltiazem.
| IIb C-LD | 2. Catheter ablation may be reasonable for asymptomatic patients with recurrent atrial flutter (102,121,372). |
| See Online Data Supplements 16 and 17. | Catheter ablation of atrial flutter is highly effective, with single-procedure success rates >90% (102) and an excellent safety profile (102,121). In patients with recurrent atrial flutter, long-term maintenance of sinus rhythm is more likely with ablation than with pharmacological therapy (372). Also, ablation may avoid potential development of tachycardia-mediated cardiomyopathy. |


**Citation [5]:** mosca-et-al-2011-effectiveness-based-guidelines-for-the-prevention-of-cardiovascular-disease-in-women-2011-update

Cardiac rehabilitation
A comprehensive CVD risk-reduction regimen such as cardiovascular or stroke rehabilitation or a physician-guided home- or community-based exercise training program should be recommended to women with a recent acute coronary syndrome or coronary revascularization, new-onset or chronic angina, recent cerebrovascular event, peripheral arterial disease (Class I; Level of Evidence A) or current/prior symptoms of heart failure and an LVEF ≤35% (Class I; Level of Evidence B).

Initialize CrewAI

In [12]:
from crewai import Agent, Task, Crew, Process
from graphlit_tools import PromptTool, CrewAIConverter
from textwrap import dedent

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

prompt_tool = CrewAIConverter.from_tool(PromptTool(graphlit))

# Embed patient profile information into the agent goals
# Define the CrewAI agents
guideline_expert = Agent(
    role="Guideline Expert",
    goal=(
        f"Retrieve and summarize relevant ACC guidelines for a {patient_profile['age']}-year-old "
        f"with atrial fibrillation and peripheral arterial disease. "
        "Focus on treatment options such as catheter ablation and antiarrhythmic therapy."
    ),
    backstory="You are an expert on ACC guidelines for managing atrial fibrillation and PAD.",
    tools=[prompt_tool],
    verbose=True,
)

patient_educator = Agent(
    role="Patient Educator",
    goal=(
        f"Translate the medical guidelines into easy-to-understand terms for Sarah Johnson. "
        "Focus on explaining the treatment options available for managing atrial fibrillation."
    ),
    backstory="You explain complex medical terms in patient-friendly language.",
    verbose=True,
)

treatment_planner = Agent(
    role="Treatment Planner",
    goal=(
        f"Create a personalized treatment plan for Sarah Johnson, considering her preference for minimally invasive procedures "
        f"and her options of catheter ablation or antiarrhythmic therapy."
    ),
    backstory="You specialize in personalized treatment plans based on patient history and preferences.",
    verbose=True,
)

output_generator = Agent(
    role="Output Generator",
    goal=(
        f"Compile the information into a comprehensive patient decision aid document for Sarah Johnson, "
        f"including a clear summary of her treatment options and next steps."
    ),
    backstory="You ensure that the medical recommendations are presented clearly and concisely.",
    verbose=True,
)

# Define tasks for each agent
task1 = Task(
    description="Retrieve ACC guidelines for managing atrial fibrillation and PAD.",
    expected_output="Summarized guidelines with a focus on catheter ablation and stroke risk reduction.",
    agent=guideline_expert,
)

task2 = Task(
    description="Translate the guidelines into patient-friendly language.",
    expected_output="Simplified, patient-friendly explanations of the treatment options.",
    agent=patient_educator,
)

task3 = Task(
    description="Personalize the treatment plan according to the patient's preferences.",
    expected_output="A treatment plan tailored to the patient's specific conditions and preferences.",
    agent=treatment_planner,
)

task4 = Task(
    description="Generate a patient decision aid document.",
    expected_output="A final decision aid document summarizing the patient's condition and treatment recommendations.",
    agent=output_generator,
)

# Create the Crew and define the process flow
crew = Crew(
    agents=[guideline_expert, patient_educator, treatment_planner, output_generator],
    tasks=[task1, task2, task3, task4],
    process=Process.sequential,  # Ensures tasks are executed in sequence
    verbose=True
)

# Execute the multi-step reasoning process
result = await crew.kickoff_async()

# Display the final patient decision aid
print("Final Patient Decision Aid:")
print(result)

# Agent: Guideline Expert
## Task: Retrieve ACC guidelines for managing atrial fibrillation and PAD.
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for PromptInput
prompt
  Input should be a valid string [type=string_type, input_value={'description': 'Retrieve... (PAD).', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Graphlit RAG prompt tool accepts these inputs: Tool Name: Graphlit RAG prompt tool
Tool Arguments: {'prompt': {'description': 'Text prompt which is provided to LLM for completion, via RAG pipeline.', 'type': 'str'}}
Tool Description: Accepts user prompt as string.
    Prompts LLM with relevant content and returns completion from RAG pipeline. Returns Markdown text from LLM completion.
    Uses vector embeddings and similarity search to retrieve relevant content from knowledge base.
    Can search through web pages, PDFs, au